# Problemas de trabajo práctico: Sesiones de doblaje
## Autor
Miguel Ángel Álvarez Cabanes
## Github
https://github.com/maalvarezcabanes/algoritmos_optimizacion

## Importación paquetes y funciones auxiliares

In [185]:
import pandas as pd
import random
import os

In [186]:
show_expanded_solution = True

## Enunciado
Se precisa coordinar el doblaje de una película. Las reglas de planificación que se deben seguir son:
1. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas.
2. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben.
3. No es posible grabar más de 6 tomas por día.
4. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.

## Lectura de datos de tomas y actores
Los datos originales están en https://docs.google.com/spreadsheets/d/1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0/edit#gid=0, pero los descargo a un fichero "doblaje.xlsx" por comodidad.

A continuación leo los datos con pandas limpiando el DataFrame para solo quedarme con los datos de actores y tomas

In [187]:
df = pd.read_excel(os.path.join(".", "doblaje.xlsx"), skiprows=1).drop(["Unnamed: 11", "Total"], axis=1)
df.set_index("Toma", inplace=True)
df.drop("TOTAL", inplace=True)
df.dropna(inplace=True)
df

,1,2,3,4,5,6,7,8,9,10
Toma,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
6,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Mínimo coste hipotético
Para establecer un mínimo coste hipotético, voy a sumar las sesiones de cada actor y lo voy a dividir entre seis. Hipotéticamente si los actores pudiesen grabar las sesiones de forma independiente, la suma de esos costes sería el mínimo. Es probable que esa solución no sea factible, pero sabemos seguro que ninguna solución va a ser mejor que esa.

In [188]:
uso_total_actores = df.sum()
display(uso_total_actores)
coste_minimo_actor = -(uso_total_actores // -6)
display(coste_minimo_actor)
coste_minimo = int(sum(coste_minimo_actor))
display(coste_minimo)

1     22.0
2     14.0
3     13.0
4     15.0
5     11.0
6      8.0
7      3.0
8      4.0
9      2.0
10     2.0
dtype: float64

1     4.0
2     3.0
3     3.0
4     3.0
5     2.0
6     2.0
7     1.0
8     1.0
9     1.0
10    1.0
dtype: float64

21

En el mejor de los casos sabemos que no vamos a conseguir un coste menor que 21.

## Planteamiento inicial: Solución naïve
Para la representación de datos voy a crear una lista de días, y cada uno de los días va a estar representado por otra lista de escenas que se ruedan ese día.

La solución naïve consistiría en elegir por orden en grupos de seis tomas por día, por lo que el máximo de días debería ser cinco. Voy a crear esta solución inicial ya que puede ser usada como una referencia de "peor solución".

Nota: Voy a tratar de hacer el código usando el menor número posible de funcionalidades de librerías para tratar de que la algoritmia usada quede lo más visible posible. Por comodidad sí usaré el DataFrame para acceder a la información.

In [189]:
def print_solucion(df, dias, coste, info_dias, show_expanded_solution = False):
    if show_expanded_solution:
        i = 1
        for dia, info in zip(dias, info_dias):
            print(f"Dia: {i}")
            print(f"Sesiones: {dia}")
            print(f"Uso actores: {info[0]}")
            print(f"Coste día: {info[1]}\n")
            i += 1
                                
    print(f"Solucion: {dias}")
    print(f"Coste total: {coste}")

In [190]:
def coste_dia(df, sesiones):
    coste = 0
    uso_actores = [sum(row) for index, row in df.loc[sesiones].T.iterrows()]
    for uso_actor in uso_actores:
        if uso_actor:
            coste += 1
    return (uso_actores, coste)

In [191]:
def coste_total(df, dias):
    coste = 0
    info_dias = []
    for num_dia, dia in enumerate(dias):
        (uso_actores, coste_aux) = coste_dia(df, dia)
        info_dias.append((uso_actores, coste_aux))
        coste += coste_aux
    return (info_dias, coste)

In [192]:
def solucion_naive(df, max_dias = 5, debug = False):
    sesiones = list(df.index)
    dias = []
    for i in range(max_dias):
        dias.append(sesiones[max_dias*i: max_dias*(i+1)])
    
    return (dias, coste_total(df, dias))

In [193]:
dias_naive, (info_dias_naive, coste_naive) = solucion_naive(df, debug=False)
print_solucion(df, dias_naive, coste_naive, info_dias_naive, show_expanded_solution = show_expanded_solution)

Dia: 1
Sesiones: [1, 2, 3, 4, 5]
Uso actores: [2.0, 4.0, 2.0, 3.0, 3.0, 0.0, 2.0, 2.0, 0.0, 0.0]
Coste día: 7

Dia: 2
Sesiones: [6, 7, 8, 9, 10]
Uso actores: [5.0, 5.0, 0.0, 3.0, 2.0, 2.0, 0.0, 0.0, 1.0, 0.0]
Coste día: 6

Dia: 3
Sesiones: [11, 12, 13, 14, 15]
Uso actores: [5.0, 3.0, 3.0, 2.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0]
Coste día: 8

Dia: 4
Sesiones: [16, 17, 18, 19, 20]
Uso actores: [3.0, 0.0, 4.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0]
Coste día: 6

Dia: 5
Sesiones: [21, 22, 23, 24, 25]
Uso actores: [3.0, 2.0, 3.0, 2.0, 0.0, 2.0, 0.0, 1.0, 0.0, 1.0]
Coste día: 7

Solucion: [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15], [16, 17, 18, 19, 20], [21, 22, 23, 24, 25]]
Coste total: 34


In [194]:
df2 = pd.DataFrame([["naïve", len(dias_naive), coste_naive]], columns = ["Algoritmo", "Numero dias", "Coste"])
df2

,Algoritmo,Numero dias,Coste
0,naïve,5,34


### Conclusiones
Como se puede observar, la solución requiere tres días de rodaje y un coste de 34 unidades.

## Aproximación voraz (I)
En esta aproximación voy a ir asignando las sesiones de forma ordenada escogiendo siempre la primera que "quepa" en el día y relegando a días posteriores las que no.

In [195]:
def chequear_factibilidad(df, sesiones_dia, nueva_sesion, limite = 6, debug = False):
    uso_actores = [sum(row) for index, row in df.loc[sesiones_dia].T.iterrows()]
    posible_uso_actores = [sum(row) for row in zip(uso_actores, list(df.loc[nueva_sesion]))]
    if debug:
        print(f"Sesiones dia: {sesiones_dia}. Posible nueva sesion: {nueva_sesion}")
        print(f"Posible uso de actores: {posible_uso_actores}. Max {max(posible_uso_actores)}")
    if max(posible_uso_actores) > limite:
        return False
    else:
        return True

In [196]:
def solucion_voraz(df, sesiones, max_dias = 5, debug = False):
    dias = []
    to_remove = []
    for i in range(max_dias):
        if debug:
            print(f"Sesiones: {sesiones}")
            print(f"Usadas en iteracion anterior: {to_remove}")
        for z in to_remove:
            sesiones.remove(z)
        to_remove = []
        for j in sesiones:
            if len(dias) < i+1:
                dias.append([j])
                to_remove.append(j)
            else:
                if chequear_factibilidad(df, dias[i], j, debug=debug):
                    dias[i].append(j)
                    to_remove.append(j)
    
    if debug:
        print(f"Solucion: {dias}")
              
    return (dias, coste_total(df, dias))

In [197]:
sesiones = list(df.index)
dias_voraz, (info_dias_voraz, coste_voraz) = solucion_voraz(df, sesiones, debug=False)
print_solucion(df, dias_voraz, coste_voraz, info_dias_voraz, show_expanded_solution = show_expanded_solution)

Dia: 1
Sesiones: [1, 2, 3, 4, 5, 6, 7, 13, 14, 18, 21, 24]
Uso actores: [6.0, 6.0, 5.0, 6.0, 6.0, 4.0, 2.0, 3.0, 0.0, 0.0]
Coste día: 8

Dia: 2
Sesiones: [8, 9, 10, 11, 12, 15, 16, 27]
Uso actores: [6.0, 6.0, 2.0, 4.0, 2.0, 3.0, 1.0, 1.0, 1.0, 1.0]
Coste día: 10

Dia: 3
Sesiones: [17, 19, 20, 22, 23, 25]
Uso actores: [6.0, 2.0, 5.0, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]
Coste día: 6

Dia: 4
Sesiones: [26, 28, 29, 30]
Uso actores: [4.0, 0.0, 1.0, 2.0, 2.0, 1.0, 0.0, 0.0, 1.0, 0.0]
Coste día: 6

Solucion: [[1, 2, 3, 4, 5, 6, 7, 13, 14, 18, 21, 24], [8, 9, 10, 11, 12, 15, 16, 27], [17, 19, 20, 22, 23, 25], [26, 28, 29, 30]]
Coste total: 30


In [198]:
df2.loc[1] = ["voraz", len(dias_voraz), coste_voraz]
df2

,Algoritmo,Numero dias,Coste
0,naïve,5,34
1,voraz,4,30


Aunque consigo una solución mejor que la "naïve", en el fondo es fácil observar que en cierta manera pasa algo similar a una aproximación voraz con el problema de la mochila (aunque en este caso minimizando en lugar de maximizando)... es fáci observar que los actores 7, 8, 9, y 10 (que son los que tienen menos carga de trabajo total) han repartido su trabajo en varios días en lugar de concentrarlo.

## Aproximación voraz (II)
Me pregunto qué pasaría si aplicamos la misma solución, pero ordenando primero las secuencias de forma que las primeras sean las que están los actores menos usados, de forma que tiendan a agruparse sus escenas.

El orden "lógico" sería 10, 9, 7, 8, 6, 5, 3, 4, 2, 1 poniendo por orden inverso de uso de actores, pero usando 10, 9, 8, 7, 6, 5, 4, 3, 2, 1 consigo 26 en lugar de 27.

In [199]:
def reordenar_sesiones(sesiones, orden_actores):
    sesiones_ordered = []
    for actor in orden_actores:
        for sesion in sesiones:
            if df.loc[sesion][actor]:
                sesiones_ordered.append(sesion)
                sesiones.remove(sesion)
    return sesiones_ordered

In [200]:
sesiones = list(df.index)
orden_actores = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
sesiones_reordenadas = reordenar_sesiones(sesiones, orden_actores)
dias_voraz2, (info_dias_voraz2, coste_voraz2) = solucion_voraz(df, sesiones_reordenadas, debug=False)
print_solucion(df, dias_voraz2, coste_voraz2, info_dias_voraz2, show_expanded_solution = show_expanded_solution)

Dia: 1
Sesiones: [16, 25, 10, 26, 4, 11, 21, 3, 15, 18, 24, 27, 2]
Uso actores: [6.0, 6.0, 5.0, 4.0, 5.0, 4.0, 3.0, 3.0, 2.0, 2.0]
Coste día: 10

Dia: 2
Sesiones: [8, 12, 14, 29, 1, 6, 5]
Uso actores: [6.0, 5.0, 3.0, 4.0, 3.0, 4.0, 0.0, 1.0, 0.0, 0.0]
Coste día: 7

Dia: 3
Sesiones: [13, 20, 7, 22, 28, 17]
Uso actores: [6.0, 2.0, 3.0, 5.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Coste día: 5

Dia: 4
Sesiones: [23, 9, 30]
Uso actores: [3.0, 1.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Coste día: 4

Solucion: [[16, 25, 10, 26, 4, 11, 21, 3, 15, 18, 24, 27, 2], [8, 12, 14, 29, 1, 6, 5], [13, 20, 7, 22, 28, 17], [23, 9, 30]]
Coste total: 26


In [201]:
df2.loc[2] = ["voraz2", len(dias_voraz2), coste_voraz2]
df2

,Algoritmo,Numero dias,Coste
0,naïve,5,34
1,voraz,4,30
2,voraz2,4,26


Creo que la utilización de este punto de partida ha sido muy acertado. Está claro que la forma en la que he implementado la voracidad está muy condicionada por el punto de partida.

## Aproximación voraz multiarranque
Esta aproximación es añternativa a la que he denominado "voraz2", ya que en lugar de ordenar de forma intencionada las sesiones con un determinado criterio, voy a buscar otros puntos de partida utilizando un multiarranque barajando el orden de inicio de las sesiones

In [202]:
def solucion_voraz_multiarranque(df, sesiones, intentos, max_dias = 5, debug = False):
    dias_voras_multi, coste_voraz_multi = dias_naive, coste_naive # :TODO: Chequear si pasarlo como parámetro
    for i in range(intentos):
        sesiones_aux = sesiones.copy()
        random.shuffle(sesiones_aux)
        if debug:
            print(f"Orden sesiones: {sesiones_aux}")
        dias_aux, (info_aux, coste_aux) = solucion_voraz(df, sesiones_aux, debug=debug)
        if debug:
            print(f"Coste voraz: {coste_aux}")
        if coste_aux < coste_voraz_multi:
            dias_voras_multi, (info_voraz_multi, coste_voraz_multi) = dias_aux, (info_aux, coste_aux)
            
    return (dias_voras_multi, (info_voraz_multi, coste_voraz_multi))

In [203]:
sesiones = list(df.index)
dias_voraz_multi, (info_dias_voraz_multi, coste_voraz_multi) = solucion_voraz_multiarranque(df, sesiones, 200, debug=False)
print_solucion(df, dias_voraz_multi, coste_voraz_multi, info_dias_voraz_multi, show_expanded_solution = show_expanded_solution)

Dia: 1
Sesiones: [6, 2, 25, 4, 1, 8, 27, 23, 3, 24, 21, 18, 16]
Uso actores: [6.0, 6.0, 5.0, 6.0, 5.0, 4.0, 2.0, 2.0, 0.0, 2.0]
Coste día: 9

Dia: 2
Sesiones: [7, 11, 13, 19, 5, 15, 9]
Uso actores: [6.0, 5.0, 2.0, 4.0, 3.0, 0.0, 1.0, 2.0, 0.0, 0.0]
Coste día: 7

Dia: 3
Sesiones: [14, 10, 12, 22, 29, 30]
Uso actores: [6.0, 3.0, 3.0, 3.0, 1.0, 4.0, 0.0, 0.0, 1.0, 0.0]
Coste día: 7

Dia: 4
Sesiones: [28, 26, 20, 17]
Uso actores: [4.0, 0.0, 3.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0]
Coste día: 5

Solucion: [[6, 2, 25, 4, 1, 8, 27, 23, 3, 24, 21, 18, 16], [7, 11, 13, 19, 5, 15, 9], [14, 10, 12, 22, 29, 30], [28, 26, 20, 17]]
Coste total: 28


In [204]:
df2.loc[3] = ["voraz_multi", len(dias_voraz_multi), coste_voraz_multi]
df2

,Algoritmo,Numero dias,Coste
0,naïve,5,34
1,voraz,4,30
2,voraz2,4,26
3,voraz_multi,4,28


Ni siquiera con un número "alto" de arranques (200) consigo resultados mejores que los de la ordenación inicial dirigida que he llamado "voraz2". En el fondo tiene sentido, ya que estoy permutando 30 elementos, por lo que el espacio de posibilidades es 30! La posibilidad de encontrarme con una ordenación inicial interesante es mínima

## Aproximación con búsqueda local de actores
Como he mencionado antes al reordenar las secuencias según el uso de actores, he conseguido mejores resultados con una ordenación que no era exactamente la que habría sido "lógica" usando una ordenación por uso de actores. Sin embargo, curiosamente la que he usado al final podría haberse obtenido como una exploración local de la otra.

Voy a explorar algo más esa idea implementando un algoritmo de búsqueda local en ese sentido. Voy a dejar que se intercambien actores con una o dos posiciones de diferencia con respecto al orden inicial. 

In [225]:
def busqueda_local_actores(df, sesiones, orden_actores, debug = False):
    sesiones_copy = sesiones.copy()
    sesiones_reordenadas = reordenar_sesiones(sesiones_copy, orden_actores)
    dias_voraz_ref, (info_dias_voraz_ref, coste_voraz_ref) = solucion_voraz(df, sesiones_reordenadas, debug=debug)
    for j in range(2):
        for i in range(len(orden_actores)-1-j):
            orden_aux = orden_actores.copy()
            orden_aux[i], orden_aux[i+1+j] = orden_aux[i+1+j], orden_aux[i]
            sesiones_copy = sesiones.copy()
            sesiones_reordenadas = reordenar_sesiones(sesiones_copy, orden_aux)
            dias_voraz_aux, (info_dias_voraz_aux, coste_voraz_aux) = solucion_voraz(df, sesiones_reordenadas, debug=debug)
            if coste_voraz_aux < coste_voraz_ref:
                print(f"Nuevo orden de actores: {orden_aux}")
                dias_voraz_ref, (info_dias_voraz_ref, coste_voraz_ref) = dias_voraz_aux, (info_dias_voraz_aux, coste_voraz_aux)
                orden_actores = orden_aux
    return (dias_voraz_ref, (info_dias_voraz_ref, coste_voraz_ref))

In [227]:
sesiones = list(df.index)
#orden_actores = [10, 9, 7, 8, 6, 5, 3, 4, 2, 1]
orden_actores = [7, 8, 9, 10, 6, 5, 4, 3, 2, 1]
dias_local, (info_dias_local, coste_local) = busqueda_local_actores(df, sesiones, orden_actores, debug=False)
print_solucion(df, dias_local, coste_local, info_dias_local, show_expanded_solution = show_expanded_solution)

Nuevo orden de actores: [7, 8, 9, 10, 6, 5, 4, 1, 2, 3]
Dia: 1
Sesiones: [3, 15, 4, 11, 21, 10, 26, 16, 25, 18, 24, 27, 2]
Uso actores: [6.0, 6.0, 5.0, 4.0, 5.0, 4.0, 3.0, 3.0, 2.0, 2.0]
Coste día: 10

Dia: 2
Sesiones: [8, 12, 14, 29, 1, 6, 5]
Uso actores: [6.0, 5.0, 3.0, 4.0, 3.0, 4.0, 0.0, 1.0, 0.0, 0.0]
Coste día: 7

Dia: 3
Sesiones: [13, 20, 7, 22, 28, 9]
Uso actores: [6.0, 3.0, 2.0, 6.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Coste día: 5

Dia: 4
Sesiones: [19, 30, 17]
Uso actores: [3.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Coste día: 3

Solucion: [[3, 15, 4, 11, 21, 10, 26, 16, 25, 18, 24, 27, 2], [8, 12, 14, 29, 1, 6, 5], [13, 20, 7, 22, 28, 9], [19, 30, 17]]
Coste total: 25


In [220]:
df2.loc[4] = ["local_actores", len(dias_local), coste_local]
df2

,Algoritmo,Numero dias,Coste
0,naïve,5,34
1,voraz,4,30
2,voraz2,4,26
3,voraz_multi,4,28
4,local_actores,4,25
